In [ ]:
0.## Kết nối Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)
folder = '/content/gdrive/My Drive/KLTN'

Mounted at /content/gdrive


#THƯ VIỆN CẦN THIẾT PHỤC VỤ CHO BÀI TOÁN

In [ ]:
!pip install gradio


In [ ]:
import gradio as gr
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
import matplotlib.pyplot as plt
from keras. callbacks import EarlyStopping

#CHUẨN BỊ DỮ LIỆU CHO MÔ HÌNH

In [ ]:
df=pd.read_csv(folder+ '/Tesla.csv')
df.shape

(1692, 7)

In [ ]:
# Lấy ra các đặc trưng và chuẩn hóa dữ liệu
features = df[['Open', 'High', 'Low', 'Adj Close', 'Volume']].values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

# Hàm tạo tập dữ liệu cho mô hình (time step = 30)
def create_dataset(data, time_step=30):
    X, Y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), :])  # Chọn tất cả các cột đặc trưng
        Y.append(data[i + time_step, 3])  # Cột 3 là giá đóng cửa
    return np.array(X), np.array(Y)

# Thiết lập số bước thời gian
time_step = 30
X, Y = create_dataset(scaled_features, time_step)

#HÀM KHỞI TẠO MÔ HÌNH LR, RNN, LSTM

In [ ]:
# Hàm mô hình Linear Regression
def linear_regression_model(X_train, Y_train, X_test, Y_test):
    model = LinearRegression()
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    mse = mean_squared_error(Y_test, Y_pred)
    r2 = r2_score(Y_test, Y_pred)
    return Y_pred, mse, r2


In [ ]:
# Mô hình RNN
def rnn_model(X_train, Y_train, X_test, Y_test):
    X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2]))
    X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2]))

    model = tf.keras.Sequential([
        tf.keras.layers.SimpleRNN(50, activation='relu', return_sequences=True,
                                   input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.SimpleRNN(50, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse')
  # Sử dụng EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    model.fit(X_train_reshaped, Y_train, epochs=50, batch_size=64, verbose=0,
              validation_split=0.2, callbacks=[early_stopping])
    Y_pred = model.predict(X_test_reshaped)
    mse = mean_squared_error(Y_test, Y_pred)
    r2 = r2_score(Y_test, Y_pred)
    return Y_pred, mse, r2


In [ ]:
# Mô hình LSTM
def lstm_model(X_train, Y_train, X_test, Y_test):
    X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2]))
    X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2]))

    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(50, activation='relu', return_sequences=True,
                             input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(50, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    # Sử dụng EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    model.fit(X_train_reshaped, Y_train, epochs=50, batch_size=64, verbose=0,
              validation_split=0.2, callbacks=[early_stopping])
    Y_pred = model.predict(X_test_reshaped)

    mse = mean_squared_error(Y_test, Y_pred)
    r2 = r2_score(Y_test, Y_pred)
    return Y_pred, mse, r2

# CHIA TÁCH, HUẤN LUYỆN VÀ ĐÁNH GIÁ HIỆU SUẤT

In [ ]:
def time_series_split(model_type):
    tscv = TimeSeriesSplit(n_splits=5)  # Chia dữ liệu thành 5 lần gập
    mse_list = []
    r2_list = []
    plt.figure(figsize=(10, 6))
    for fold, (train_index, test_index) in enumerate(tscv.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        if model_type == "Linear Regression":
            Y_pred, mse, r2 = linear_regression_model(X_train, Y_train, X_test, Y_test)
        elif model_type == "LSTM":
            Y_pred, mse, r2 = lstm_model(X_train, Y_train, X_test, Y_test)
        elif model_type == "RNN":
            Y_pred, mse, r2 = rnn_model(X_train, Y_train, X_test, Y_test)
        mse_list.append(mse)
        r2_list.append(r2)
        # Đảo ngược chuẩn hóa để hiển thị kết quả thực tế
        Y_test_actual = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], Y_test.reshape(-1, 1)), axis=1))[:, -1]
        Y_pred_actual = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], Y_pred), axis=1))[:, -1]
        # Vẽ đồ thị cho fold cuối cùng (trong vòng lặp)
        if fold == tscv.get_n_splits() - 1:  # Chỉ vẽ ở lần chia cuối cùng
          plt.plot(Y_test_actual, label="Giá trị thực", color='blue')
          plt.plot(Y_pred_actual, label="Dự đoán", color='red')
          plt.title(f"Kết quả dự đoán với mô hình {model_type}")
          plt.xlabel("Thời gian")
          plt.ylabel("Giá trị")
          plt.legend()
    mse_avg = np.mean(mse_list)
    r2_avg = np.mean(r2_list)
    plt.savefig('plot.png')
    plt.close()  # Đóng để không làm lộn xộn các biểu đồ sau này
    return mse_avg, r2_avg, 'plot.png'


#THIẾT LẬP GIAO DIỆN

In [ ]:
# --- Thiết lập giao diện Gradio ---
def interface(model_type):
    mse_avg, r2_avg, plot_path = time_series_split(model_type)
    return mse_avg, r2_avg, plot_path

In [ ]:
# Tạo ứng dụng Gradio
demo = gr.Interface(
    fn=interface,
    inputs=gr.Radio(["Linear Regression", "LSTM", "RNN"], label="Chọn mô hình"),
    outputs=[gr.Number(label="MSE trung bình"), gr.Number(label="R-squared trung bình"), gr.Image(label="Đồ thị dự đoán")],
    title="So sánh các mô hình dự đoán với TimeSeriesSplit",
    description="Chọn một mô hình để xem kết quả dự đoán và các chỉ số đánh giá"
)

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6527f8fcfee6a61a5f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
